In [1]:
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger

import nltk

nltk.download('stopwords')

stop_words= nltk.corpus.stopwords.words('russian')

/home/user/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [9]:
def lemmatize_text(text_data):
    """
    Функция для лемматизации текста
    """

    doc = Doc(text_data)

    doc.segment(segmenter)

    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return [_.lemma for _ in doc.tokens]

def delete_stop_words(text_data, stop_words):
    """
    Функция для удаления стоп-слов
    """
    return [word for word in text_data if not word in stop_words]


def delete_number_and_signs_words(text_data):
    """
    Функция для удаления ненужных символов
    """
    alphabet = ("а","б","в","г","д","е","ё","ж","з","и","й","к","л","м","н","о",
            "п","р","с","т","у","ф","х","ц","ч","ш","щ","ъ","ы","ь","э","ю","я")
    return [word for word in text_data if  
                              all(letter in alphabet for letter in word)]

def preserve_nouns(text_data):
    doc = Doc(" ".join(text_data))
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    return [token.text for token in doc.tokens if token.pos=='NOUN']
    
    

In [56]:
HtmlFile = open('messages.html', 'r', encoding='utf-8')
source_code = HtmlFile.read() 

In [59]:
from bs4 import BeautifulSoup

In [67]:
 
parsed_html = BeautifulSoup(source_code)

In [96]:
messages = parsed_html.find_all('div', attrs={'class':'text'})[1:]

In [125]:
texts = [message.text for message in messages]
dst_text = texts[0]
print(dst_text)


Добро пожаловать в официальный телеграм-канал оператора электронных торгов «Росэлторг»!Здесь вас ждут:✅ актуальные новости компании: новые продукты, нововведения в уже существующих, анонсы мероприятий✅ новости отрасли: что происходит в мире закупок и как с этим работать✅ лайфхаки для заказчиков и поставщиков: разбираем законодательство, делимся гайдами по закупкам и ответами на ваши вопросы от экспертов✅ интересные закупки, опубликованные на площадке «Росэлторг»Подписывайтесь и следите за миром закупок вместе с нами!
       


In [126]:
for i, text in enumerate(texts):
    lemmatized_text = lemmatize_text(text)
    sw_d_text = delete_stop_words(lemmatized_text, stop_words)
    d_nas_text = delete_number_and_signs_words(sw_d_text)
    nouns_text = preserve_nouns(d_nas_text)
    texts[i] = " ".join(nouns_text)

In [127]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)


In [132]:
dst_tfidf =  X.toarray()[0]

In [133]:
ind2v = {i: dst_tfidf[i] for i in range(len(dst_tfidf))}

In [134]:
ind2v_sorted = dict(sorted(ind2v.items(), key=lambda item: item[1], reverse=True))

In [136]:
p = 1.6
sum_ = 0.
i = 0
lst = []
while sum_ < p and i < len(ind2v_sorted):
    sum_ += list(ind2v_sorted.values())[i]
    lst.append(list(ind2v_sorted.keys())[i])
    i += 1
vectorizer.get_feature_names_out()[lst].tolist()    

['мир', 'новость', 'закупка', 'анонс', 'гайда']